# 1) Libraries & Toolkits 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings

warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize':(12,8)} ,font_scale=1.2)

# 2) Importing Dataset and exploration

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
print(train.shape)
train.head()

In [ ]:
test  = pd.read_csv('../input/titanic/test.csv')
print(test.shape)
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
# - Number of uniques values in dataframe

pd.DataFrame({'': train.nunique().index , 'Number of unique values':  train.nunique().values})

In [ ]:
train.describe().T

# 3) Exploratory data analysis

In [ ]:
train[train.Sex == 'male' ]['Survived'].value_counts(normalize=True).mul(100).round(0).astype(str) + ' %'

In [ ]:
train[train.Sex == 'female' ]['Survived'].value_counts(normalize=True).mul(100).round(0).astype(str) + ' %'

In [ ]:
sns.countplot(data=train, x='Survived', hue='Sex' , palette='Set2')

In [ ]:
sns.countplot(data=train, hue='Pclass', x='Survived', palette='Set2')

In [ ]:
sns.catplot(data=train, x='Survived', hue='Sex', kind='count', col='Pclass' , palette='Set2')

In [ ]:
sns.countplot(data=train, x='Embarked', hue='Pclass' , palette='Set2')

In [ ]:
sns.kdeplot(train['Age'], shade=True, color='g')

# 4) Pre-processing & Feature Engineering

In [ ]:
## Fill Nan Values
train['Age'] = train['Age'].fillna(round(train['Age'].mean()))
test['Age'] = test['Age'].fillna(round(test['Age'].mean()))
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].value_counts().idxmax())
test['Fare'] = test['Fare'].fillna(test['Fare'].value_counts().idxmax())


In [ ]:
def title(name):
    return name.split(',')[1].split('.')[0]

def age_categories(age):
    if age <= 3:
        return 'Infant'
    elif age <= 14 :
        return 'Child'
    elif age <= 28 :
        return 'Youth'
    elif age <= 60 :
        return 'Adult'
    else :
        return 'Old Adult'


In [ ]:
train['Title'] = train['Name'].apply(title)
test['Title'] = test['Name'].apply(title)

In [ ]:
train['Alone'] = train['SibSp'] + train['Parch']
test['Alone'] = test['SibSp'] + test['Parch']
train.Alone = train.Alone.apply(lambda x : 'Yes' if x == 0 else 'No')
test.Alone = test.Alone.apply(lambda x : 'Yes' if x == 0 else 'No')


In [ ]:
train['Age_Group'] = train['Age'].apply(age_categories)
test['Age_Group'] = test['Age'].apply(age_categories)
train = train.drop(columns=['Name' , 'Cabin' , 'PassengerId'])
test = test.drop(columns=['Name' , 'Cabin'])
train.head()

In [ ]:
test.head()

In [ ]:
lbl=LabelEncoder()
data = [train , test]
nums = []
for dataset in data:
    for col in dataset.loc[ : , dataset.dtypes == np.object].columns:
        dataset[col] = lbl.fit_transform(dataset[col])
        directory = {label: index for index , label  in enumerate(lbl.classes_)}
        nums.append(directory)
nums[0]

In [ ]:
train.sample(5)

In [ ]:
test.sample(5)

# 5) Modeling

In [ ]:
x_train = train.drop("Survived", axis=1)
y_train = train[["Survived"]]
x_test  = test.drop(columns=['PassengerId']).copy()

x_train.shape, y_train.shape, x_test.shape 

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

model = rfc.fit(x_train , y_train)
pred = model.predict(x_test)

print("Accuracy on Test Data : {:,.2f}".format(rfc.score(x_train , y_train) *100 ) , '%')

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": pred
    })
submission

In [ ]:
# submission.to_csv('submission.csv', index=False)